In [2]:
import json
import unicodedata
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/gmaps_details.csv", index_col=0
)

idx = df[~df["details_output_from_candidate_request"].isna()].index
df["Website"] = None

for i in df.index:
    try:
        df.loc[i, "Website"] = json.loads(
            df.loc[i, "details_output_from_candidate_request"]
        )["result"]["website"]
    except:
        df.loc[i, "Website"] = None


df.drop(
    columns=[
        "SICCode.SicText_1",
        "SICCode.SicText_2",
        "SICCode.SicText_3",
        "SICCode.SicText_4",
        "is_referenced",
        "CompanyStatus",
    ],
    inplace=True,
)
websites_list = df[~df["Website"].isna()]["Website"].unique()

In [9]:
def get_text_func1(url):

    html = urlopen(url, timeout=30).read()
    soup = BeautifulSoup(html, features="lxml")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = " ".join(chunk for chunk in chunks if chunk)
    text = unicodedata.normalize("NFKD", text)

    return text

In [ ]:
cpt_total = 0
cpt_err = 0

for i in html.index:
    cpt_total += 1
    print(i)
    html.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv")
    try:
        html.loc[i, "Content"] = get_text_func1(html.loc[i, "Website"])
    except:
        html.loc[i, "Content"] = "Error"
        cpt_err += 1
        print(round(cpt_err / cpt_total * 100, 2))

In [ ]:
# html = pd.DataFrame(websites_list)
# html.rename(columns = {0:'Website'}, inplace = True)
# html['Content'] = None
# html.to_csv('/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv')

In [ ]:
cpt_total = 0
cpt_err = 0
html = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv", index_col=0
)
idx_to_process = html[html["Content"].isna()].index

for i in idx_to_process:
    cpt_total += 1
    print(i)

    html.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv")
    try:

        html.loc[i, "Content"] = get_text_func1(html.loc[i, "Website"])

    except:
        html.loc[i, "Content"] = "Error"
        cpt_err += 1
        print(round(cpt_err / cpt_total * 100, 2))

In [3]:
# html = pd.DataFrame(websites_list)
# html.rename(columns = {0:'Website'}, inplace = True)
# html['Content'] = None
# html.to_csv('/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv')

In [ ]:
cpt_total = 0
cpt_err = 0
html = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv", index_col=0
)
idx_to_process = html[
    (html["Content"].isna() == True) | (html["Content"] == "Error")
].index

for i in idx_to_process:
    cpt_total += 1
    print(i)

    html.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv")
    try:

        html.loc[i, "Content"] = get_text_func1(html.loc[i, "Website"])

    except:
        html.loc[i, "Content"] = "Error"
        cpt_err += 1
        print(round(cpt_err / cpt_total * 100, 2))

In [17]:
import urllib.request

from bs4.element import Comment


def tag_visible(element):
    if element.parent.name in [
        "style",
        "script",
        "head",
        "title",
        "meta",
        "[document]",
    ]:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, "html.parser")
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return " ".join(t.strip() for t in visible_texts)


html = urllib.request.urlopen(websites_list[0]).read()
print(text_from_html(html))

     01908676767 Home Book Cab Online About Us Services Cranfield Taxi Olney Taxi Silverstone Taxi Woburn Taxi Blog F.a.q Contact Us Testimonial  Book Online & Get 10% Off Book Cab Online  Call Now 01908676767  Email Now 247taxilinemk  WhatsApp Now +44 7960 931946 Airport Transfers: Milton Keynes Taxi Service Near Me | Cab Hire Online Choose Your Car Mercedes Taxi Wheel Chair Taxi VW Transport Vito Taxi Saloon Taxi  Base Rate: £3  Book Cab  Per Mile: £1.90  Book Cab Airport Transfers Milton Keynes With years of experience and a large fleet of luxurious taxis, we have your diverse commuting needs sorted. 24/7 Taxi Line is a well-established taxi service that continues to grow with its ever-expanding customer-base in Milton Keynes and its surrounding areas. While you have an important meeting scheduled on a short notice, it is easy to find yourself in a state of panic and looking for a taxi online, using the most common ‘taxi near me’ phrase. Since 24/7 Taxi Line operates with its extens

## Second attempt

In [48]:
import urllib.request



def tag_visible(element):
    if element.parent.name in [
        "style",
        "script",
        "head",
        "title",
        "meta",
        "[document]",
    ]:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(url):
    body = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(body, "lxml")
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return " ".join(t.strip() for t in visible_texts)


text_from_html(websites_list[0])

"     01908676767 Home Book Cab Online About Us Services Cranfield Taxi Olney Taxi Silverstone Taxi Woburn Taxi Blog F.a.q Contact Us Testimonial  Book Online & Get 10% Off Book Cab Online  Call Now 01908676767  Email Now 247taxilinemk  WhatsApp Now +44 7960 931946 Airport Transfers: Milton Keynes Taxi Service Near Me | Cab Hire Online Choose Your Car Mercedes Taxi Wheel Chair Taxi VW Transport Vito Taxi Saloon Taxi  Base Rate: £3  Book Cab  Per Mile: £1.90  Book Cab Airport Transfers Milton Keynes With years of experience and a large fleet of luxurious taxis, we have your diverse commuting needs sorted. 24/7 Taxi Line is a well-established taxi service that continues to grow with its ever-expanding customer-base in Milton Keynes and its surrounding areas. While you have an important meeting scheduled on a short notice, it is easy to find yourself in a state of panic and looking for a taxi online, using the most common ‘taxi near me’ phrase. Since 24/7 Taxi Line operates with its exten

In [10]:
df = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/gmaps_details.csv", index_col=0
)

idx = df[~df["details_output_from_candidate_request"].isna()].index
df["Website"] = None

for i in df.index:
    try:
        df.loc[i, "Website"] = json.loads(
            df.loc[i, "details_output_from_candidate_request"]
        )["result"]["website"]
    except:
        df.loc[i, "Website"] = None


df.drop(
    columns=[
        "SICCode.SicText_1",
        "SICCode.SicText_2",
        "SICCode.SicText_3",
        "SICCode.SicText_4",
        "is_referenced",
        "CompanyStatus",
    ],
    inplace=True,
)
websites_list = df[~df["Website"].isna()]["Website"].unique()

In [18]:
html = pd.DataFrame(websites_list)
html.rename(columns={0: "Website"}, inplace=True)
html["Content"] = None
html.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv")

In [46]:
html.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '2712', '2713', '2714', '2715', '2716', '2717', '2718', '2719', '2720',
       '2721'],
      dtype='object', length=2723)

In [24]:
cpt_total = 0
cpt_err = 0

html = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv", index_col=0
)
idx_to_process = html[html["Content"] == "Error"].index

for i in idx_to_process:
    cpt_total += 1
    print(i)

    html.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv")
    try:

        html.loc[i, "Content"] = text_from_html(html.loc[i, "Website"])

    except:
        html.loc[i, "Content"] = "Error"
        cpt_err += 1
        print(round(cpt_err / cpt_total * 100, 2))

5
100.0
18
100.0
19
100.0
26
100.0
28
100.0
31
100.0
39
100.0
42
100.0
53
54
90.0
58
90.91
62
91.67
66
92.31
82
92.86
89
93.33
90
93.75
91
94.12
99
94.44
101
94.74
104
95.0
107
95.24
108
95.45
113
95.65
119
95.83
121
96.0
132
96.15
143
96.3
146
96.43
147
96.55
152
96.67
167
96.77
170
96.88
177
96.97
181
97.06
183
97.14
185
97.22
189
97.3
193
97.37
199
97.44
206
97.5
209
97.56
222
97.62
226
97.67
227
97.73
230
97.78
244
97.83
250
97.87
252
97.92
262
97.96
275
98.0
279
98.04
280
98.08
283
98.11
289
98.15
290
98.18
292
98.21
296
98.25
307
98.28
312
98.31
340
98.33
346
98.36
349
98.39
354
98.41
356
98.44
360
98.46
362
98.48
363
98.51
374
98.53
378
98.55
389
98.57
394
98.59
395
98.61
396
98.63
397
98.65
398
98.67
399
98.68
400
98.7
401
98.72
407
98.73
409
98.75
411
98.77
415
98.78
419
98.8
425
98.81
427
98.82
431
98.84
433
98.85
434
98.86
435
98.88
437
98.89
451
98.9
454
98.91
463
98.92
466
98.94
469
98.95
471
98.96
476
98.97
478
98.98
483
98.99
486
99.0
490
99.01
491
99.02
496
99.03
501
99

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


626
97.67
634
97.69
635
97.71
639
97.73
648
97.74
655
97.76
664
97.78
670
97.79
676
97.81
679
97.83
681
97.84
682
97.86
708
97.87
714
97.89
716
97.9
720
97.92
726
97.93
736
97.95
738
97.96
743
97.97
745
97.99
747
98.0
757
98.01
762
763
97.39
766
97.4
772
97.42
781
97.44
783
97.45
802
97.47
804
97.48
809
97.5
813
97.52
819
97.53
828
97.55
833
97.56
839
97.58
841
97.59
843
97.6
847
97.62
853
97.63
859
97.65
860
97.66
871
97.67
872
97.69
873
97.7
875
97.71
883
97.73
884
97.74
892
97.75
900
97.77
908
97.78
915
97.79
925
97.8
931
97.81
932
97.83
944
97.84
947
97.85
955
97.86
956
97.87
967
97.88
969
97.89
971
97.91
979
97.92
980
97.93
981
97.94
982
97.95
983
97.96
990
97.97
991
97.98
993
97.99
995
98.0
1006
98.01
1026
98.02
1029
98.03
1035
98.04
1045
98.05
1047
98.06
1048
98.07
1051
98.08
1056
98.09
1057
98.1
1066
98.1
1070
98.11
1071
98.12
1072
98.13
1074
98.14
1075
98.15
1078
98.16
1087
98.17
1098
98.17
1104
98.18
1107
98.19
1115
98.2
1122
98.21
1130
98.21
1132
98.22
1134
98.23
1135
98.24


## Idem avec Crunchbase!


In [2]:
df = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/CB_2.csv", index_col=0
)
websites_list = df[~df["Website"].isna()]["Website"].unique()
len(websites_list)

467

In [3]:
"""html_cb = pd.DataFrame(websites_list)
html_cb.rename(columns = {0:'Website'}, inplace = True)
html_cb['Content'] = None
html_cb.to_csv('/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv')"""

In [ ]:
cpt_total = 0
cpt_err = 0

html_cb = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv", index_col=0
)
idx_to_process = html_cb[
    (html_cb["Content"].isna() == True) | (html_cb["Content"] == "Error")
].index

for i in idx_to_process:
    cpt_total += 1
    print(i)

    html_cb.to_csv("/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv")
    try:

        html_cb.loc[i, "Content"] = get_text_func1(
            "https://www." + html_cb.loc[i, "Website"]
        )
    except:
        html_cb.loc[i, "Content"] = "Error"
        cpt_err += 1
        print(round(cpt_err / cpt_total * 100, 2))

4
100.0
5
100.0
6
100.0
8
100.0
12
100.0
15
100.0
18
100.0
19
100.0
21
100.0
24
100.0
25
100.0
26
100.0
29
100.0
30
100.0
31
100.0
32
100.0
36
100.0
37
100.0
38
100.0
40
100.0
47
100.0
49
100.0
51
100.0
55
100.0
56
100.0
57
100.0
58
100.0
59
100.0
61
100.0
63
100.0
66
100.0
75
100.0
83
100.0
86
100.0
91
100.0
92
100.0
94
100.0
95
100.0
99
100.0
102
100.0
103
100.0
105
100.0
106
100.0
107
100.0
109
100.0
112
100.0
114
100.0
115
100.0
116
100.0
117
100.0
122
100.0
127
100.0
129
100.0
134
100.0
135
100.0
136
100.0
137
100.0
139
100.0
140
100.0
142
100.0
144
100.0
146
100.0
150
100.0
153
100.0
158
100.0
159
100.0
164
100.0
170


In [15]:
html_cb["Content"]

0      Homebase - DIY That Turns Your House Into A Ho...
1      Envisics - We redefine the way people see and ...
2      AMN Search this site HomeAbout UsAbout UsMissi...
3      Automotive Technology Consultancy and Research...
4                                                  Error
                             ...                        
462                                                Error
463                                                Error
464    Badminton England | The Nation’s Favourite Rac...
465    Pathfinders in cyber intelligence | CYSIAM Ski...
466    Kinetic - Student Accommodation and Event Mana...
Name: Content, Length: 467, dtype: object

In [14]:
idx_to_process = html_cb[
    (html_cb["Content"].isna() == True) | (html_cb["Content"] == "Error")
].index
len(idx_to_process)

144

In [83]:
html_cb = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv", index_col=0
)
idx_to_process = html_cb[
    (html_cb["Content"].isna() == True) | html_cb["Content"] == "Error"
].index

In [84]:
html_cb["Content"].unique()

array(['Error', nan], dtype=object)

In [4]:
html_cb = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv", index_col=0
)
# idx_to_drop = html_cb[(html_cb['Content'].isna()==True) | (html_cb['Content'] =='Error')].index

html_cb["Content"]

0      Homebase - DIY That Turns Your House Into A Ho...
1      Envisics - We redefine the way people see and ...
2      AMN Search this site HomeAbout UsAbout UsMissi...
3      Automotive Technology Consultancy and Research...
4                                                  Error
                             ...                        
462                                                Error
463                                                Error
464    Badminton England | The Nation’s Favourite Rac...
465    Pathfinders in cyber intelligence | CYSIAM Ski...
466                                                  NaN
Name: Content, Length: 467, dtype: object

In [12]:
"""Hypertext: Documents present on the web are
different from general text-only documents
because of the presence of hyperlinks. It is
estimated that there are roughly 10 hyperlinks
present per document.
 Heterogeneity of document: The contents present
on a web page are heterogeneous in nature i.e., in
addition to text they might contain other
multimedia contents like audio, video and images.
 Duplication: On the Web, over 20% of the
documents present are either near or exact
duplicates of other documents and this estimation
has not included the semantic duplicates yet.
 Lack of stability: Web pages lack stability in the
sense that the contents of Web pages are modified
frequently. """.replace(
    "\n", " "
)

'Hypertext: Documents present on the web are different from general text-only documents because of the presence of hyperlinks. It is estimated that there are roughly 10 hyperlinks present per document. \uf0b7 Heterogeneity of document: The contents present on a web page are heterogeneous in nature i.e., in addition to text they might contain other multimedia contents like audio, video and images. \uf0b7 Duplication: On the Web, over 20% of the documents present are either near or exact duplicates of other documents and this estimation has not included the semantic duplicates yet. \uf0b7 Lack of stability: Web pages lack stability in the sense that the contents of Web pages are modified frequently. '